In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

- Interceptor: txt database
- we take till rank 200 and not first 200 rows
- find vs findall
- NYU issue
- Indian Institute of Science (IISc) Bangalore issue
- json['data'][0].keys()

In [2]:
def find_div_and_number(soup, class_name):
    
    div = soup.find('div', class_= class_name)
    if div is not None:
        return div.find('div', class_='number').text.replace('\n', '').replace(',', '')
    else: return np.NaN
        

def parse_uni_details(url):
    page_body = requests.get('https://topuniversities.com' + uni['url']).text

    soup = BeautifulSoup(page_body, "html.parser")
    
    faculty_total = find_div_and_number(soup, 'total faculty')
    faculty_intl = faculty_total = find_div_and_number(soup, 'inter faculty')
    student_total = faculty_total = find_div_and_number(soup, 'total student')
    student_intl = faculty_total = find_div_and_number(soup, 'total inter')
    
    return faculty_total, faculty_intl, student_total, student_intl

In [3]:
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508153509501')
json = r.json()

topuniversities = pd.DataFrame()

for (idx, uni) in enumerate(json['data']):
    
    if int(uni['rank_display'].replace('=','')) > 200:
        break
        
    faculty_total, faculty_intl, student_total, student_intl = parse_uni_details(uni['url'])
        
    entry = pd.DataFrame({'name': uni['title'], 'rank': uni['rank_display'].replace('=', ''), 'country': uni['country'],
                          'region': uni['region'], 'faculty_intl': faculty_intl, 'faculty_total': faculty_total,
                          'student_intl': student_intl, 'student_total': student_total}, index=[idx]) 
    topuniversities = topuniversities.append(entry)

# Rearrange columns order
topuniversities = topuniversities[['name', 'rank', 'country', 'region', 'faculty_intl', 'faculty_total', 'student_intl', 'student_total']]

# Change rank, faculty and student cols to numeric value
topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']]\
    = topuniversities[['rank','faculty_intl', 'faculty_total', 'student_intl', 'student_total']].apply(pd.to_numeric)
    
# Serialization as a pickle file
topuniversities.to_pickle('pickled/topuniversities.pkl')
    
topuniversities.head()

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,3717.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,3611.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,5266.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,647.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,6699.0,6699.0,18770.0


In [4]:
topuniversities = pd.read_pickle('pickled/topuniversities.pkl')
topuniversities.head()

,name,rank,country,region,faculty_intl,faculty_total,student_intl,student_total
0,Massachusetts Institute of Technology (MIT),1,United States,North America,1679.0,3717.0,3717.0,11067.0
1,Stanford University,2,United States,North America,2042.0,3611.0,3611.0,15878.0
2,Harvard University,3,United States,North America,1311.0,5266.0,5266.0,22429.0
3,California Institute of Technology (Caltech),4,United States,North America,350.0,647.0,647.0,2255.0
4,University of Cambridge,5,United Kingdom,Europe,2278.0,6699.0,6699.0,18770.0
